Task 1: Write a function result = central_moment(image, i, j) that returns the (i, j) central moment of image, where image is a grayscale or binary image.

Task 2: Write a function result = normalized_moment(image, i, j) that returns the (i, j) normalized central moment of image, where image is a grayscale or binary image.

Task 3: Write a function result = hu_moment(image, m) that returns the m-th Hu moment of image, where image is a grayscale or binary image. Note that the lecture slides define 7 Hu moments.

Task 4: Write a function prediction = knn_classify_digit_cm(digit, K, train_cmoments_db) that takes as input an image of a digit from the MNIST dataset and the number of neighbors (K) to use, and it predicts the label of the digit by performing a nearest neighbor search using central moments on the train_cmoments_db.
The train_cmoments_db contains a database of precomputed central moments of 5000 digits. Each vector of central moments contains the following eight central moments <m00, m11, m20, m02, m21, m12, m30, m03> in that order. The database also contains the labels and the minimum and maximum value of any moment that was observed in the training set before they were normalized to the range [0, 1].
You can extract the moment vectors, labels, min values and max values as:
(train_cmoments, train_labels, min_vals, max_vals) = train_cmoments_db
The algorithmic steps to implement the function are as follows:
1. Compute a feature vector of the same central moments from the query digit.
2. Normalize each moment to the range [0, 1], by using the min_vals and max_vals from the train_cmoments_db.
3. Compute the Euclidean distance of the digit feature vector from each of the 5000 vectors in the database.
4. Find the top K nearest neighbors.
5. Assign the predicted label of the digit to be that of the majority of its top K neighbors. 
 
We will test your function by predicting the labels of digits 5001 to 6000 (1000 digits) from the MNIST database and comparing them to the true label. My solution predicts 747/1000 labels correctly for K=1 and 795/1000 for K=11. To get full credit, your solution should correctly predict at least 700/1000 digits. 5 out of the 40 points of this task will be allocated to code efficiency in terms of running time. To get full credit, your solution should take no more than three times as much as my solution for classifying the 1000 digits.
 
Tips:
The Euclidean distance between two feature vectors can be negatively affected if the range of values of one feature is much different than the range of values of another feature. To avoid such problems, we can scale all features to the range [0, 1]. In my solution, scaling improved the prediction accuracy by about 5%.
Computing the Euclidean distance between the query digit feature vector and each of the 5000 feature vectors of our database in a loop is slow. Instead of a loop, use Numpy matrix operations to subtract the current moments vector from each row of the train_cmoments matrix and to compute all the distances in one operation.
When computing the Euclidean distance, make sure to scale the query feature vector using the same min and max values that were calculated from the database feature vectors.

In [1]:
#main operations
import os
import sys
import numpy as np
import time
from sklearn.metrics import confusion_matrix
from get_features_cm import get_features_cm
from knn_classify_digit_cm import knn_classify_digit_cm

data_file_path = "data/mnist_data.csv"
################# Load the data #################
# Load the data. Use ',' as the delimiter
data = np.loadtxt(data_file_path, delimiter=',')
labels = data[:, 0]
data = data[:, 1:]
# Reshape the data to be a list of 28x28 2D images
data = data.reshape(data.shape[0], 28, 28)

# Split the data into training and test sets
train_data = data[:5000]
train_labels = labels[:5000]

test_data = data[5000:6000]
test_labels = labels[5000:6000]
########## Compute the central moments of training data ##########
train_cmoments = np.zeros((train_data.shape[0], 8))
for i in range(train_data.shape[0]):
    train_cmoments[i] = get_features_cm(train_data[i])

# Scale all features of the database to the range [0, 1]
min_vals = np.min(train_cmoments, axis=0)
max_vals = np.max(train_cmoments, axis=0)
train_cmoments = (train_cmoments - min_vals) / (max_vals - min_vals)

train_cmoments_db = (train_cmoments, train_labels, min_vals, max_vals)


################# Classify test data #################
# Predict the labels of all test digits

# Time the prediction process
start_time = time.time()

predictions = np.zeros(len(test_labels))
K = 11
for i in range(len(test_labels)):
    # Predict the digit using the KNN classifier
    digit = test_data[i,:,:]
    predictions[i] = knn_classify_digit_cm(digit, K, train_cmoments_db)

end_time = time.time()
elapsed_time = end_time - start_time
print('\nTime elapsed for prediction: {} seconds'.format(elapsed_time))

# Calculate the accuracy
correct = np.sum(predictions == test_labels)
total = len(test_labels)
accuracy = correct / total

print('\nCorrectly predicted {} out of {} digits'.format(correct, total))
print('Accuracy: {}'.format(accuracy))

# Print confusion matrix
print('\nConfusion matrix:')
print(confusion_matrix(test_labels, predictions))


Time elapsed for prediction: 11.831400632858276 seconds

Correctly predicted 795 out of 1000 digits
Accuracy: 0.795

Confusion matrix:
[[103   0   1   2   0   0   0   0   2   0]
 [  1 110   0   0   0   1   2   0   1   0]
 [  4   0  84   2   0   1   2   0   2   0]
 [  2   0  13  67   0   5   1   1   5   1]
 [  4   0   0   0  73   2   4   0   0  16]
 [ 18   1   7  11   1  39   1   0  13   1]
 [  2   0   2   0   0   0  96   0   0   0]
 [  0   0   0   1   0   0   0  80   1  15]
 [ 12   2   3   1   4  15   2   2  57   0]
 [  0   0   0   0   0   0   0  13   2  86]]
